In [1]:
from itertools import product, permutations, pairwise
from functools import cache

In [ ]:
with open("input-example.txt") as f:
    lines = f.readlines()
lines = [l.strip() for l in lines]
lines[:5]

In [3]:
numpad = [
    "789",
    "456",
    "123",
    "#0A",
]

directional_pad = [
    "#^A",
    "<v>",
]

In [4]:
dirs = [(-1, 0), (0, 1), (1, 0), (0, -1)]
dir_map = {"^": (-1, 0), ">": (0, 1), "v": (1, 0), "<": (0, -1)}


def walk(pos, dir):
    return (pos[0] + dir[0], pos[1] + dir[1])


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def look_ahead(pos, dir, field):
    next_step = walk(pos, dir)
    if not in_bounds(next_step, field):
        return "."
    return field[next_step[0]][next_step[1]]


def in_bounds(pos, field):
    return (
        pos[0] >= 0 and pos[0] < len(field) and pos[1] >= 0 and pos[1] < len(field[0])
    )


def print_field(field, robot=None):
    i = 0
    for l in field:
        j = 0
        for p in l:
            if robot and robot[0] == i and robot[1] == j:
                print("@", end="")
            else:
                print(p, end="")
            j += 1
        print()
        i += 1


# Search the first position of a key in a field when looking from top left to bottom right row wise
def get_pos(key, field) -> tuple[int, int]:
    for i in range(0, len(field)):
        for j in range(0, len(field[0])):
            # print(field[i][j])
            if field[i][j] == key:
                return (i, j)


# check if following a set of keys from a position hits a forbidden position (blank position in this case)
def hits_evil(pos1, keys, evil_pos):
    cur_pos = pos1
    # print(cur_pos, keys, evil_pos)
    for k in keys:
        cur_pos = walk(cur_pos, dir_map[k])
        # print(cur_pos)
        if cur_pos == evil_pos:
            return True
        elif k == "A":
            continue

    return False


# Print a keycombination result on a given keypad for debugging
def print_key_combination(keycomb, pad):
    if len(pad) == 2:
        cur_pos = (0, 2)
    else:
        cur_pos = (3, 2)
    for k in keycomb:
        if k == "A":
            print(pad[cur_pos[0]][cur_pos[1]], end="")
        else:
            cur_pos = walk(cur_pos, dir_map[k])
    print()

In [5]:
# Compute for the numpad what directional pad buttons would need to be pressed to reach every field from every field without hitting the blank spot
numpad_keys = "A0123456789"
numpad_combinations = product(numpad_keys, repeat=2)
numpad_from_to = {}
for comb in numpad_combinations:
    res_str = ""
    if comb[0] == comb[1]:
        all_allowed_mv_strs = [""]
    else:
        # print(comb)
        pos1 = get_pos(comb[0], numpad)
        pos2 = get_pos(comb[1], numpad)
        diff = (pos2[0] - pos1[0], pos2[1] - pos1[1])

        left_right_str = ">" * diff[1] if diff[1] >= 0 else "<" * abs(diff[1])
        up_down_str = "v" * diff[0] if diff[0] >= 0 else "^" * abs(diff[0])
        mv_str = up_down_str + left_right_str
        # Need to use set here, as both e.g. << permutations are the same to use
        all_mv_strs = list(set(permutations(mv_str)))
        all_allowed_mv_strs = [
            mvs for mvs in all_mv_strs if not hits_evil(pos1, mvs, (3, 0))
        ]

    res_str_list = ["".join(mvs) + "A" for mvs in all_allowed_mv_strs]
    numpad_from_to[comb] = res_str_list
# numpad_from_to

In [6]:
# Compute the same for the directional keypad
directional_keys = "A<>v^"
directional_combinations = product(directional_keys, repeat=2)
directional_from_to = {}
for comb in directional_combinations:
    res_str = ""
    if comb[0] == comb[1]:
        all_allowed_mv_strs = [""]
    else:
        pos1 = get_pos(comb[0], directional_pad)
        pos2 = get_pos(comb[1], directional_pad)
        diff = (pos2[0] - pos1[0], pos2[1] - pos1[1])

        left_right_str = ">" * diff[1] if diff[1] >= 0 else "<" * abs(diff[1])
        up_down_str = "v" * diff[0] if diff[0] >= 0 else "^" * abs(diff[0])
        mv_str = up_down_str + left_right_str
        all_mv_strs = list(set(permutations(mv_str)))
        all_allowed_mv_strs = [
            mvs for mvs in all_mv_strs if not hits_evil(pos1, mvs, (0, 0))
        ]

    res_str_list = ["".join(mvs) + "A" for mvs in all_allowed_mv_strs]
    # print(res_str)
    directional_from_to[comb] = res_str_list
# directional_from_to

In [ ]:
# Hardcoded expanding all pads and keeping all paths that are possible
# This works okay for part 1, part 2 needed something else
res_sum = 0
for line in lines:
    # print(line)
    current_numpad_pos = "A"
    numpad_res_strs = [""]
    for c in line:
        possible_moves = numpad_from_to[(current_numpad_pos, c)]
        temp_numpad_res_strs = []
        for pm in possible_moves:
            temp_numpad_res_strs += [s + pm for s in numpad_res_strs]
        numpad_res_strs = temp_numpad_res_strs
        current_numpad_pos = c
    # print(numpad_res_strs)

    dirpad1_res_strs_total = []
    for rs in numpad_res_strs:
        current_dirpad1_pos = "A"
        dirpad1_res_strs = [""]
        for c in rs:
            possible_moves = directional_from_to[(current_dirpad1_pos, c)]
            temp_dirpad1_res_strs = []
            for pm in possible_moves:
                temp_dirpad1_res_strs += [s + pm for s in dirpad1_res_strs]
            dirpad1_res_strs = temp_dirpad1_res_strs
            current_dirpad1_pos = c
        dirpad1_res_strs_total += dirpad1_res_strs
    # print(dirpad1_res_strs_total)

    dirpad2_res_strs_total = []
    for rs in dirpad1_res_strs_total:
        current_dirpad2_pos = "A"
        dirpad2_res_strs = [""]
        for c in rs:
            possible_moves = directional_from_to[(current_dirpad2_pos, c)]
            temp_dirpad2_res_strs = []
            for pm in possible_moves:
                temp_dirpad2_res_strs += [s + pm for s in dirpad2_res_strs]
            dirpad2_res_strs = temp_dirpad2_res_strs
            current_dirpad2_pos = c
        dirpad2_res_strs_total += dirpad2_res_strs

    min_len = min([len(s) for s in dirpad2_res_strs_total])
    # print(int(line.replace("A","")),min_len)
    res_sum += int(line.replace("A", "")) * min_len


res_sum

In [8]:
# The trick on this day is to realize that the length is only dependant on the current movement between entered characters in the inital code (and all subsequent) and the level we are on for the pad
# So finding the minimum length of keystrokes between two characters and summing them up for all pairs of two chars works recursively.

# Lets say we only look for code "4", so moving from A to 4
# This means two left and two up and pressing A, six possible ways to do that, but <<^^A is not allowed as it would go over the blank, so 5 left
# Depending on the order of the 4 movement pairs, it sometimes costs 11 and sometimes costs 15 inputs on the level above
# Turning on all prints below makes that more clear

# Since we start counting from 0, boundary 3 is 4 levels: the numerical pad, two robot directional pads and my directional pad


@cache
def search_path(keys, level, boundary=3):
    # print(keys)
    if level == boundary:
        # print("Returning: ", len(keys))
        return len(keys)

    keys = "A" + keys
    from_to_pairs = list(pairwise(keys))
    # print(from_to_pairs)
    res = 0
    for pair in from_to_pairs:
        # print("Searching pair: ", p)

        # Choose possible paths to move within the pair from our precomputed dicts depending on the level
        if level == 0:
            paths = numpad_from_to[pair]
        else:
            paths = directional_from_to[pair]
        rec_search = [search_path(path, level + 1, boundary) for path in paths]
        # print(rec_search)
        res += min(rec_search)
    # print(res)
    return res


# search_path("4", 0,boundary=2)
# search_path('^<<^A', 1,boundary=2)

In [ ]:
s = 0
for line in lines:
    print(line)
    s += int(line.replace("A", "")) * search_path(line, 0, boundary=3)
s

In [ ]:
s = 0
for line in lines:
    print(line)
    s += int(line.replace("A", "")) * search_path(line, 0, boundary=26)
s

In [11]:
# print_key_combination('v<A<AA>>^AAvA<^A>AAvA^Av<A>^A<A>Av<A>^A<A>Av<A<A>>^AAvA<^A>A', directional_pad)
# print_key_combination('v<<AA>^AA>AvA^AvA^Av<AA>^A', directional_pad)
# print_key_combination("<<^^A>A>AvvA", numpad)